In [ ]:
from collections import defaultdict
import pickle
import pandas as pd
import numpy as np

df = pd.read_csv("data/wordbank_instrument_data.csv")
df

In [ ]:
# item_kindがwordのものだけを抽出
# valueを文字列から数字に置き換える

df_word = df[df["item_kind"] == "word"]
df_word = df_word.replace({"value": {"produces": 1, "understands": 1}})
# df_word = df_word.replace({"value": {"understands": 0}})
df_word = df_word.fillna({"value": 0})
df_word = df_word.reset_index(drop=True)
df_word

In [ ]:
word_count = 680
df_category = df_word[:word_count][["category", "item_id", "item_definition"]]
# df_category["category"].value_counts()

word_dict = defaultdict(list)
category_dict = defaultdict(list)
for category, item_id, item_definition in zip(
    df_category["category"], df_category["item_id"], df_category["item_definition"]
):
    word_dict[int(item_id[5:]) - 1] = (category, item_definition)
    category_dict[category].append((int(item_id[5:]) - 1, item_definition))
word_dict = dict(word_dict)
category_dict = dict(category_dict)

In [ ]:
print(word_dict)  # {VAEの入力ノードの何番目か(0-index): (category, item_definition), }
print(
    category_dict
)  # {category: [(VAEの入力ノードの何番目か(0-index), item_definition), ], }

In [ ]:
word_count = 680
group_number = np.arange(len(df_word)) // word_count
correct_item_ids = [f"item_{i+1}" for i in range(word_count)]

data_tmp = []
data_id_dict = {}
child_id_dict = defaultdict(list)
for i, group in df_word.groupby(group_number):
    if (group["child_id"] == group.iat[0, 8]).all() and (
        group["item_id"] == correct_item_ids
    ).all():
        vocabulary = group["value"].to_numpy(dtype=np.float32)
        data_id_dict[i] = (
            group.iat[0, 8],
            group.iat[0, 9],
            vocabulary.sum(),
        )
        child_id_dict[(group.iat[0, 8])].append((i, group.iat[0, 9], vocabulary.sum()))

        data_tmp.append(vocabulary)

data = np.array(data_tmp)
child_id_dict = dict(child_id_dict)

In [ ]:
print(data_id_dict) # {data_id: (child_id, age, vocabulary), }
print(child_id_dict) # {child_id: [(data_id, age, vocabulary), (data_id, age, vocabulary), ], }
for key, val in child_id_dict.items():
    if len(val) >= 2:
        print(key, val)

In [ ]:
print(data)
print(data.shape)

In [ ]:
# 変数を保存
with open("tmp/data.pkl", "wb") as f:
    pickle.dump(data, f)
with open("tmp/data_id_dict.pkl", "wb") as f:
    pickle.dump(data_id_dict, f)
with open("tmp/child_id_dict.pkl", "wb") as f:
    pickle.dump(child_id_dict, f)
with open("tmp/word_dict.pkl", "wb") as f:
    pickle.dump(word_dict, f)
with open("tmp/category_dict.pkl", "wb") as f:
    pickle.dump(category_dict, f)